### Use the aligned data, and our previous MLE-like counting technique to find out the cases in which the same written token corresponds to two different levels



In [21]:
from google.colab import drive
import pandas as pd
import os
drive.mount('/content/drive')


base_path_aligned = '../data/readability_data'

train_aligned = pd.read_csv(base_path_aligned + '/train_pnx.tsv', sep = '\t')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
train_aligned

,Word,Label,Book,Chapter,Line
0,الفصل,3,0,2.0,0
1,الثاني,3,0,2.0,0
2,«,3,0,2.0,1
3,وكان,3,0,2.0,1
4,صباح,3,0,2.0,1
...,...,...,...,...,...
129740,إلى,3,14,8.0,10
129741,ما,3,14,8.0,10
129742,بعد,3,14,8.0,10
129743,العشاء,3,14,8.0,10


In [23]:
def get_level_counts_aligned(words, levels):
  dict_levels = {}
  for word, level in zip(words, levels):
      try:
          #assume every entry of dict_levels : {3: int, 4: int, 5: int}
          dict_levels[word][level] += 1
      except:
          dict_levels[word] = {3: 0, 4: 0, 5: 0}
          dict_levels[word][level] += 1
  return dict_levels

def counting_pipeline_aligned(data):
  counts = get_level_counts_aligned(data['Word'], data['Label'])
  return counts

In [24]:
counts = counting_pipeline_aligned(train_aligned)

In [25]:
duplicates = []
threes = []
fours = []
fives = []

for x in counts.items():
  # only 5, 4, or 3. single out those with ambiguous leveling
  if x[1][3] != 0 and x[1][4] == 0 and x[1][5] == 0:
    threes.append(x)
  elif x[1][3] == 0 and x[1][4] != 0 and x[1][5] == 0:
    fours.append(x)
  elif x[1][3] == 0 and x[1][4] == 0 and x[1][5] != 0:
    fives.append(x)
  else:
    # more than one level
    duplicates.append(x)





In [26]:
len(duplicates)

881

In [27]:
len(counts)

31940

In [28]:
_5and4 = [d for d in duplicates if d[1][5] != 0 and d[1][4] !=0 and d[1][3] == 0]
_5and3 = [d for d in duplicates if d[1][5] != 0 and d[1][4] ==0 and d[1][3] != 0]
_4and3 = [d for d in duplicates if d[1][5] == 0 and d[1][4] !=0 and d[1][3] != 0]
_all = [d for d in duplicates if d[1][5] != 0 and d[1][4] !=0 and d[1][3] != 0]

In [29]:
print(len(_5and4), len(_5and3), len(_4and3), len(_all))

89 253 473 66


In [30]:
print(len(threes), len(fours), len(fives))

22260 5423 3376
